In [1]:
import ast
import os
from flask import Flask, render_template, request
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import nltk
from nltk.corpus import stopwords

In [2]:
DATA_FILE = "../data/RAW_recipes.csv"

In [4]:
df =pd.read_csv(DATA_FILE)
df.shape

(231637, 12)

In [3]:
df = pd.read_csv("../data/RAW_recipes.csv")
list_columns = ['tags','nutrition','steps','ingredients']
for col in list_columns:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else [] )
    

df.head(5)

KeyboardInterrupt: 

In [ ]:
df['ingredients'] = df['ingredients'].apply(lambda lst: [i.lower().strip() for i in lst])

In [ ]:
df.sample(5)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
182831,scottish apple orange marmalade pie,140227,60,89831,2005-10-05,"[60-minutes-or-less, time-to-make, course, mai...","[3543.5, 211.0, 1195.0, 107.0, 65.0, 181.0, 18...",10,"[set oven to 375 degrees , and set oven rack t...","this is an easy no-fail apple pie to make, and...","[pie pastry, granny smith apples, sugar, ginge...",8
81705,fakeout healthy mac and cheese,350093,20,653538,2009-01-16,"[30-minutes-or-less, time-to-make, course, mai...","[333.1, 4.0, 16.0, 0.0, 23.0, 2.0, 21.0]",5,"[boil the pasta al dente , drain, meanwhile , ...",this healthy mac has some earthy tones since i...,"[pasta, winter squash, onion, portabella mushr...",8
174907,roast pumpkin and onion soup,406417,85,1018029,2010-01-02,"[time-to-make, course, main-ingredient, prepar...","[160.1, 10.0, 52.0, 0.0, 4.0, 5.0, 8.0]",9,[chop pumpkin into chunks and place on baking ...,this is a nice soup for a special occasion whe...,"[pumpkin, honey, olive oil, onion, chili, bay ...",12
76270,easy decadent truffles,148569,120,232971,2005-12-18,"[time-to-make, course, main-ingredient, prepar...","[77.1, 5.0, 41.0, 0.0, 1.0, 11.0, 3.0]",10,"[in a large bowl , beat cream cheese until smo...",these are awesome and perfect for holiday gift...,"[cream cheese, confectioners' sugar, semi-swee...",4
167020,pumpkin dumplings gnocchi di zucca alla versil...,185959,15,108846,2006-09-13,"[15-minutes-or-less, time-to-make, course, cui...","[445.7, 19.0, 4.0, 9.0, 29.0, 34.0, 22.0]",8,"[put the cubed pumpkin into a deep saucepan , ...",tasty dumplings tossed with butter and parmesa...,"[pumpkin, water, salt, eggs, all-purpose white...",7


In [ ]:
# SUBSTITUTIONS = {
#     'basil': 'oregano',
#     'butter': 'olive oil',
#     'honey': 'maple syrup',
#     'salt': 'soy sauce',
#     'pepper': 'chili flakes'
# }

# def compute_match(user_ingredients, recipe_ingredients):
#     user_set = set([ing.lower().strip() for ing in user_ingredients])
#     recipe_set = set(recipe_ingredients)
#     if not recipe_set:
#         return 0, set(), set()
#     intersection = user_set.intersection(recipe_set)
#     missing = recipe_set - user_set
#     match_percent = (len(intersection) / len(recipe_set)) * 100
#     return match_percent, intersection, missing

In [ ]:
df.sample(10)

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
90260,garlic hamburgers for 2,265248,20,429816,2007-11-13,"[30-minutes-or-less, time-to-make, main-ingred...","[470.1, 46.0, 10.0, 15.0, 50.0, 72.0, 7.0]",10,"[shape meat in 2 patties 1 / 2 inch thick, coo...",hamburgers cooked in garlic butter. try these ...,"[ground beef, garlic cloves, butter, hamburger...",4
167028,pumpkin figs pie,42314,60,41430,2002-10-06,"[60-minutes-or-less, time-to-make, course, mai...","[166.7, 11.0, 35.0, 4.0, 4.0, 9.0, 8.0]",8,"[preheat the oven to 350 degrees f, place the ...","light and naturally sweet pie made with pears,...","[unbaked 9-inch pie crust, pears, pumpkin pure...",5
205064,swedish red cabbage,42059,45,6258,2002-10-02,"[60-minutes-or-less, time-to-make, course, mai...","[179.2, 9.0, 87.0, 15.0, 4.0, 18.0, 10.0]",7,[melt butter in a heavy large dutch oven over ...,the explosion of flavors in this dish of cabba...,"[butter, green apples, onion, red cabbage, lig...",11
198973,star sugar cookies,271947,18,311484,2007-12-14,"[30-minutes-or-less, time-to-make, course, pre...","[214.4, 16.0, 50.0, 7.0, 5.0, 13.0, 9.0]",6,"[mix together shortening , sugar , eggs and va...",i found a recipe book hidden in my room and wa...,"[shortening, sugar, eggs, vanilla, all-purpose...",7
105876,hoisin pork,47644,40,52282,2002-11-24,"[60-minutes-or-less, time-to-make, course, mai...","[390.0, 15.0, 19.0, 36.0, 102.0, 14.0, 7.0]",5,"[combine marinade ingredients in small bowl, p...",i always get compliments when i serve this. ma...,"[hoisin sauce, rice wine vinegar, soy sauce, g...",7
141916,my homemade hamburger helper,264188,35,303700,2007-11-07,"[60-minutes-or-less, time-to-make, course, mai...","[459.6, 21.0, 30.0, 21.0, 60.0, 27.0, 18.0]",4,"[brown beef with onions , garlic and pepper an...",you can make this as spicy as you wish by addi...,"[lean ground beef, onion, garlic cloves, green...",12
50030,chocolate chip refrigerator cookies,118734,22,69474,2005-04-23,"[30-minutes-or-less, time-to-make, course, pre...","[102.3, 10.0, 23.0, 1.0, 2.0, 12.0, 3.0]",27,"[in a medium bowl , thoroughly stir together t...","these are simple, and very, very good...they h...","[all-purpose white flour, baking powder, salt,...",11
114195,james river corn pudding,387296,45,192581,2009-08-26,"[60-minutes-or-less, time-to-make, course, mai...","[164.0, 14.0, 7.0, 12.0, 13.0, 24.0, 5.0]",13,"[preheat oven to 400 degrees, butter a 1 1 / 2...",this recipe was in today's nytimes food sectio...,"[butter, eggs, all-purpose flour, milk, fresh ...",7
55039,cleaning and seasoning cast iron cookware,316257,5,689540,2008-07-29,"[15-minutes-or-less, time-to-make, preparation...","[113.2, 19.0, 0.0, 1178.0, 0.0, 16.0, 0.0]",8,"[after using your cast iron skillet , if it do...",this method has worked for me for years! i ha...,"[salt, crisco shortening]",2
168344,quick and easy gluten free chicken noodle soup,264880,40,604084,2007-11-12,"[60-minutes-or-less, time-to-make, course, mai...","[355.9, 36.0, 2.0, 11.0, 44.0, 62.0, 4.0]",16,"[in a large pot , melt 2 tablespoons of the bu...",i came up with this when i started making chic...,"[chicken breasts, butter, onion, tapioca flour...",11
